# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [140]:
 # imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display, update_display
import requests
from bs4 import BeautifulSoup



In [141]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
Google API Key not set


In [142]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [143]:
system_message = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [164]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()
LLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [145]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [146]:
def get_info_web(url):
    Website(url)

In [147]:
web_function = {
    "name": "get_info_web",
    "description": "Get the information of website to explain to user. Call this whenever you need to know about the any website, for example when a user asks 'what about this website ,or could you give information about this website'",
    "parameters": {
        "type": "object",
        "properties": {
            "website_link": {
                "type": "string",
                "description": "the website that customer ask to know information about website",
            },
        },
        "required": ["website_link"],
        "additionalProperties": False
    }
}

In [148]:
tools = [{"type": "function", "function": web_function}]

In [159]:
def handle_tool_call(message):
    try:
        tool_call = message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        url = args.get('website_link')

        if not url:
            raise ValueError("Website link not provided in the tool call arguments")

        if not url.startswith(('http://', 'https://')):
            url = f"https://{url}"

        website = Website(url)
        web_info = {
            "title": website.title,
            "text": website.text,
            "links": website.links
        }

        response = {
            "role": "tool",
            "content": json.dumps({"web_info": web_info}),
            "tool_call_id": tool_call.id
        }
        return response, url 

    except Exception as e:
        print(f"Error handling tool call: {str(e)}")
        return {}, None


In [213]:
def chat_gpt(message, history): 
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        response, url = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL_GPT, messages=messages) 
    
    return response.choices[0].message.content

In [216]:
import ollama

def chat_llama(message, history):
    client = ollama.Client()
    # Constructing the messages history for the API request
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    request_payload = {
        "messages": messages,
        "model": MODEL_LLAMA
    }
    
    try:
        # Using request_payload in the API call
        response = client.chat(**request_payload)
        # Assuming response from ollama.Client().chat() is already a dict
        print("API Response:", response)

        if 'choices' in response and response['choices'][0].get('finish_reason') == "tool_calls":
            tool_message = response['choices'][0]['message']
            print("Handling tool call with message:", tool_message)
            response_message, url = handle_tool_call(tool_message)
            messages.append({"role": "system", "content": response_message})
            # Update the request payload with the new history
            request_payload['messages'] = messages
            response = client.chat(**request_payload)
            response = response  # Assuming direct use of response if dict

        return response['message']['content']

    except Exception as e:
        print("Failed to process API call:", e)
        return "Error processing your request."



In [227]:
def chat(message, history, model):
    print(model)
    if model == "GPT":
        return chat_gpt(message, history)
    elif model == "LLama":
        return chat_llama(message, history)
    else:
        return "Model not recognized."
    

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [235]:
Models = ["GPT", "LLama"] 
with gr.Blocks() as view:
    # Dropdown for model selection
    model_select = gr.Dropdown(Models, label="Select Model", value="GPT")

    chat_interface = gr.ChatInterface(
        fn=lambda message, history: chat(message, history, "GPT"), 
        type="messages"
    )

    # Function to update the ChatInterface function dynamically
    def update_chat_model(model):
        chat_interface.fn = lambda message, history: chat(message, history, model)

    # Ensure the function updates when the dropdown changes
    model_select.change(fn=update_chat_model, inputs=model_select)

    # Add the components to the Blocks view
    view.add(model_select)
    view.add(chat_interface)

view.launch()

* Running on local URL:  http://127.0.0.1:7947

To create a public link, set `share=True` in `launch()`.


LLama
API Response: model='llama3.2' created_at='2025-03-28T03:17:58.3651071Z' done=True done_reason='stop' total_duration=1682458000 load_duration=54845900 prompt_eval_count=72 prompt_eval_duration=6315300 eval_count=84 eval_duration=1619506600 message=Message(role='assistant', content="## Getting Started\nThis conversation has just begun. I'll wait for you to provide more information about the website you'd like me to analyze.\n\nIf you need my help with something specific or would like to analyze a website, please let me know by providing the URL of the website or the content you'd like me to summarize. \n\nFor example: `# Analyze this website: https://www.example.com`", images=None, tool_calls=None)
GPT
GPT
LLama
API Response: model='llama3.2' created_at='2025-03-28T03:18:26.8038878Z' done=True done_reason='stop' total_duration=2109343800 load_duration=59065100 prompt_eval_count=262 prompt_eval_duration=286861800 eval_count=113 eval_duration=1757850900 message=Message(role='assista